# 라이브러리 import

In [2]:
import pandas as pd
from pandas_gbq import read_gbq
import ast
from itertools import combinations
import os

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession 

import dotenv
from google.oauth2 import service_account

# 데이터 불러오기

## pandas

In [3]:
# BigQuery에 업로드된 데이터 불러오기 (pandas)
def load_data_from_bigquery(query):
    try:
        df = read_gbq(query,
                      project_id=dotenv.get_key(dotenv.find_dotenv(), "GOOGLE_PROJECT_ID"),
                      dialect='standard'
                      )
        return df
    except Exception as e:
        print(f"Error loading data from BigQuery: {e}")
        return None

In [4]:
votes_tables = load_data_from_bigquery("SELECT table_name FROM `votes.INFORMATION_SCHEMA.TABLES`;")

Downloading: 100%|██████████|


In [5]:
hackle_tables = load_data_from_bigquery("SELECT table_name FROM `hackle.INFORMATION_SCHEMA.TABLES`;")

Downloading: 100%|██████████|


## spark

In [3]:
# BigQuery에 업로드된 데이터 불러오기 (spark)
def load_data_from_bigquery_spark(query):
    spark = SparkSession.builder \
        .appName("BigQuery to Spark") \
        .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.24.0") \
        .getOrCreate()
    try:
        credentials = service_account.Credentials.from_service_account_file(
            dotenv.get_key(dotenv.find_dotenv(), "GOOGLE_APPLICATION_CREDENTIALS")
        )
        df = spark.read.format("bigquery") \
            .option("credentials", credentials) \
            .option("query", query) \
            .load()
        return df
    except Exception as e:
        print(f"Error loading data from BigQuery with Spark: {e}")
        return None

In [4]:
votes_tables = load_data_from_bigquery_spark("SELECT table_name FROM `votes.INFORMATION_SCHEMA.TABLES`;")

KeyboardInterrupt: 

In [ ]:
hackle_tables = load_data_from_bigquery_spark("SELECT table_name FROM `hackle.INFORMATION_SCHEMA.TABLES`;")

# 기본 통계량 함수 작성란

In [6]:
# 기본 통계량 (데이터의 크기, 타입, 결측치, describe 내용 확인) 출력 함수
def basic_information(df):
    print(f'data shape : {df.shape}')
    
    basic_info_df = pd.DataFrame(data=df.dtypes, columns=['dtypes'])
    basic_info_df['missing'] = df.isnull().sum().values
    basic_info_df['missing_rate (%)'] = df.isnull().sum().values / len(df) * 100
    basic_info_df['nunique'] = df.nunique().values

    # describe 통계량 (수치형/범주형 포함)
    desc_df = df.describe(include='all').transpose()

    # 수치형 항목만 해당되는 통계량 추가
    for col in ['mean', 'std', 'min','25%', '50%', '75%', 'max']:
        if col in desc_df.columns:
            basic_info_df[col] = desc_df[col]

    # 최빈값 및 해당 최빈값의 count
    basic_info_df['mode'] = {col: df[col].mode(dropna=True).iloc[0] if not df[col].mode(dropna=True).empty else None for col in df.columns}
    basic_info_df['mode_counts'] = {col: df[col].value_counts().max() if not df[col].value_counts().empty else 0 for col in df.columns}

    return basic_info_df

# votes 데이터베이스 확인

## head 출력

In [7]:
# votes 테이블의 기본 정보 출력
for table in votes_tables['table_name']:
    print(f"Table: {table}")
    query = f"SELECT * FROM `votes.{table}` LIMIT 5;"
    display(load_data_from_bigquery(query))

Table: polls_question
Downloading: 100%|██████████|


,id,question_text,created_at
0,99,가장 신비한 매력이 있는 사람은?,2023-03-31 15:22:53
1,100,"""이 사람으로 한 번 살아보고 싶다"" 하는 사람은?",2023-03-31 15:22:53
2,101,미래의 틱톡커는?,2023-03-31 15:22:54
3,102,여기서 제일 특이한 친구는?,2023-03-31 15:22:54
4,103,가장 지켜주고 싶은 사람은?,2023-03-31 15:22:55


Table: accounts_attendance
Downloading: 100%|██████████|


,id,attendance_date_list,user_id
0,7923,"[""2023-05-27"", ""2023-05-28"", ""2023-05-29"", ""20...",1355335
1,38921,"[""2023-05-27"", ""2023-05-28"", ""2023-05-29"", ""20...",892265
2,777,"[""2023-05-27"", ""2023-05-28"", ""2023-05-29"", ""20...",913054
3,6557,"[""2023-05-27"", ""2023-05-28"", ""2023-05-29"", ""20...",947077
4,2534,"[""2023-05-27"", ""2023-05-28"", ""2023-05-29"", ""20...",1233366


Table: polls_questionpiece
Downloading: 100%|██████████|


...

,id,distance,nearby_school_id,school_id
0,119021,0.004564,6,7
1,119022,0.010787,13,7
2,119023,0.012928,20,7
3,119024,0.013590,4,7
4,119025,0.014122,24,7


Table: polls_usercandidate
Downloading: 100%|██████████|


,id,created_at,question_piece_id,user_id
0,3088872,2023-04-28 12:27:49,998458,849444
1,3088873,2023-04-28 12:27:49,998458,849454
2,3088874,2023-04-28 12:27:49,998458,849460
3,3088875,2023-04-28 12:27:49,998458,849469
4,3088964,2023-04-28 12:28:02,998459,849446


Table: accounts_timelinereport
Downloading: 100%|██████████|


,id,reason,created_at,reported_user_id,user_id,user_question_record_id
0,4794,광고,2023-05-10 14:27:44,917587,1084275,18272641
1,7168,광고,2023-05-12 14:54:46,967873,1184397,28992803
2,7169,광고,2023-05-12 14:54:52,967873,1184397,28992803
3,9998,광고,2023-05-14 16:49:58,1207784,1214760,46645583
4,10257,광고,2023-05-15 06:08:04,1250642,1228033,48820574


## 기본 정보 출력

In [8]:
# votes 테이블의 요약, 결측치, 타입 등 기본 정보 출력
for table in votes_tables['table_name']:
    print(f"Table: {table}")
    query = f"SELECT * FROM `votes.{table}`;"
    display(basic_information(load_data_from_bigquery(query)))

Table: polls_question
Downloading: 100%|██████████|
data shape : (5025, 3)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,5025,2619.897512,1452.438472,99.0,1365.0,2621.0,3877.0,5133.0,99,1
question_text,object,0,0.0,3903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vote,56
created_at,datetime64[us],0,0.0,644,2023-05-26 17:49:11.666268,NaN,2023-03-31 15:22:53,2023-05-15 14:03:03,2023-06-02 08:06:53,2023-06-06 06:10:14,2023-06-06 06:15:52,2023-06-06 06:15:42,82


Table: accounts_attendance
Downloading: 100%|██████████|
data shape : (349637, 3)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,349637,180550.143772,104168.322237,1.0,90432.0,180749.0,270674.0,360505.0,1,1
attendance_date_list,object,0,0.0,128837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],20945
user_id,Int64,0,0.0,349637,1247187.288568,212798.63273,832151.0,1065813.0,1261234.0,1437486.0,1583730.0,832151,1


Table: polls_questionpiece
Downloading: 100%|██████████|
data shape : (1265476, 5)


...

,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,59500,148770.5,17176.314845,119021.0,133895.75,148770.5,163645.25,178520.0,119021.0,1
distance,float64,0,0.0,32006,0.055338,0.604024,0.0,0.011484,0.021658,0.064555,49.296594,0.0,6214
nearby_school_id,Int64,0,0.0,5950,2976.695882,1706.965231,4.0,1518.0,2973.0,4448.0,5964.0,4497.0,22
school_id,Int64,0,0.0,5950,2980.524538,1718.663078,4.0,1493.0,2980.5,4469.0,5964.0,4.0,10


Table: polls_usercandidate
Downloading: 100%|██████████|
data shape : (4769609, 4)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,4769609,238868397.391241,184489263.091847,3088872.0,62703391.0,212670022.0,380021511.0,646672584.0,3088872,1
created_at,datetime64[us],0,0.0,917071,2023-05-17 13:57:52.437727,NaN,2023-04-28 12:27:49,2023-05-10 02:49:19,2023-05-15 16:15:12,2023-05-22 11:33:33,2024-05-08 01:36:18,2023-05-17 13:45:43,44
question_piece_id,Int64,0,0.0,1217589,74302481.038724,57619589.580503,998458.0,18916946.0,66396553.0,117887593.0,208351468.0,9494270,16
user_id,Int64,0,0.0,19994,1090240.982971,201819.002037,833112.0,883590.0,1087460.0,1233969.0,1580629.0,860304,4226


Table: accounts_timelinereport
Downloading: 100%|██████████|
data shape : (208, 6)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,208,9786.269231,5713.767759,28.0,6123.75,10095.0,14182.25,23108.0,28,1
reason,object,0,0.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,허위 사실 언급,80
created_at,datetime64[us],0,0.0,208,2023-05-15 08:13:50.278846,NaN,2023-05-06 04:44:57,2023-05-11 16:41:43.500000,2023-05-15 01:23:29,2023-05-18 18:01:18,2023-06-02 08:30:06,2023-05-06 04:44:57,1
reported_user_id,Int64,0,0.0,125,1132496.875,169412.066968,837235.0,966470.25,1144432.0,1225963.0,1562736.0,1138757,15
user_id,Int64,0,0.0,115,1159945.668269,180095.938183,834089.0,967120.25,1184397.0,1291813.0,1548002.0,1343904,19
user_question_record_id,Int64,0,0.0,163,49325579.326923,33112807.364359,3920588.0,24611783.5,47403268.0,71795781.0,140765885.0,78776644,14


# hackle 데이터베이스 확인

## head 출력

In [9]:
# hackle 테이블의 기본 정보 출력
for table in hackle_tables['table_name']:
    print(f"Table: {table}")
    query = f"SELECT * FROM `hackle.{table}` LIMIT 5;"
    display(load_data_from_bigquery(query))

Table: device_properties
Downloading: 100%|██████████|


,id,device_id,device_model,device_vendor
0,178797,b5a029f8-39fb-487f-8607-0ff2a65f1fe7,Z4PRO,APEX
1,220605,e0034fb8-d4a2-4c28-9c9b-fab4c4762f69,Z4PRO,APEX
2,75889,4d1b5295-29ba-4606-b4bd-7f455ade35fd,Tab_MyPen_P1,Alldocube
3,34028,228937fd-0675-4ec9-99f2-d8d7460e34d7,KFMAWI,Amazon
4,43876,2c797959-01fd-4f45-8b3a-1dc34fec4951,KFMAWI,Amazon


Table: user_properties
Downloading: 100%|██████████|


,user_id,class,gender,grade,school_id
0,1236848,1,M,1,1
1,1437875,1,M,1,1
2,1572211,1,M,1,1
3,1577131,1,M,1,1
4,1577930,1,M,1,1


Table: hackle_events
Downloading: 100%|██████████|


,event_id,event_datetime,event_key,session_id,id,item_name,page_name,friend_count,votes_count,heart_balance,question_id
0,a4490697-42d9-47ad-abde-5f05a57629f8,2023-08-06 22:03:23,$session_start,5d45bf15-82eb-47e8-917c-f779ca2c35bc,a4490697-42d9-47ad-abde-5f05a57629f8,,,NaN,NaN,NaN,NaN
1,a4496ff1-1584-4705-92f3-e971fdec2de1,2023-08-06 06:02:04,$session_end,7Yy9L6IvlZWO6YQyGYLhthRj29p1,a4496ff1-1584-4705-92f3-e971fdec2de1,,,NaN,NaN,NaN,NaN
2,a45070d1-b1fa-4dcd-a369-42fb17a5271b,2023-08-06 20:33:22,launch_app,4BAAE5D5-FCEE-4995-B65C-822B69B0CD8D,a45070d1-b1fa-4dcd-a369-42fb17a5271b,,,NaN,NaN,NaN,NaN
3,a4530ae5-373f-4f0d-b89a-8d6de24455e3,2023-07-29 18:28:41,$session_start,0lwba1Cg9GP3ScQGQ7sdOe5ws5n1,a4530ae5-373f-4f0d-b89a-8d6de24455e3,,,NaN,NaN,NaN,NaN
4,a453db5c-cc95-4ea4-b47a-458a177dbce7,2023-07-18 22:13:34,launch_app,CE87B688-4149-4D9D-AAA9-6558CC27D4A0,a453db5c-cc95-4ea4-b47a-458a177dbce7,,,NaN,NaN,NaN,NaN


Table: hackle_properties
Downloading: 100%|██████████|


,id,session_id,user_id,language,osname,osversion,versionname,device_id
0,225279,aa2f0ec2-6e64-4937-88c0-139dc1e6ba3a,,ko,Android,11,1.0.0,aa2f0ec2-6e64-4937-88c0-139dc1e6ba3a
1,150431,f54427dc-87fc-4a71-9f22-351ebc3d83de,,ko,Android,11,1.2.1,f54427dc-87fc-4a71-9f22-351ebc3d83de
2,212190,e24fae5a-0896-4149-ac68-775ecd89c90e,,ko,Android,11,1.2.1,e24fae5a-0896-4149-ac68-775ecd89c90e
3,347659,752SqR0xcyWltcAkQzGs8ZNhvju1,,ko,Android,10,1.2.10,7b1d355a-1b31-42a8-bddc-d9f54a81e49b
4,360622,0c155764-4c9d-4e66-92c4-e6c1080c83f9,,ko,Android,10,1.2.10,0c155764-4c9d-4e66-92c4-e6c1080c83f9


## 기본 정보 출력

In [10]:
# hackle 테이블의 요약, 결측치, 타입 등 기본 정보 출력
for table in hackle_tables['table_name']:
    print(f"Table: {table}")
    query = f"SELECT * FROM `hackle.{table}`;"
    display(basic_information(load_data_from_bigquery(query)))

Table: device_properties
Downloading: 100%|██████████|
data shape : (252380, 4)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,252380,126190.5,72855.974806,1.0,63095.75,126190.5,189285.25,252380.0,1,1
device_id,object,0,0.0,251720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0006bed8-512b-48f9-9b33-f79bb1e225a8,2
device_model,object,0,0.0,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"iPhone14,5",27656
device_vendor,object,0,0.0,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Apple,174444


Table: user_properties
Downloading: 100%|██████████|
data shape : (230819, 5)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
user_id,object,0,0.0,230819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000,1
class,Int64,0,0.0,20,4.594999,3.151979,1.0,2.0,4.0,6.0,20.0,1,36580
gender,object,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,132610
grade,Int64,0,0.0,3,2.002197,0.762048,1.0,1.0,2.0,3.0,3.0,2,96778
school_id,Int64,0,0.0,5023,3083.383335,1711.671162,1.0,1594.0,3138.0,4640.0,5964.0,4917,206


Table: hackle_events


C:\Users\lento\AppData\Local\Temp\ipykernel_16612\3313574774.py:4: LargeResultsWarning: Recommendation: Your results are 1.8 GiB. Consider using BigQuery DataFrames (https://bit.ly/bigframes-intro)to process large results with pandas compatible APIs with transparent SQL pushdown to BigQuery engine. This provides an opportunity to save on costs and improve performance. Please reach out to bigframes-feedback@google.com with any questions or concerns. To disable this message, run warnings.simplefilter('ignore', category=pandas_gbq.exceptions.LargeResultsWarning)
  df = read_gbq(query, project_id=dotenv.get_key(dotenv.find_dotenv(), "GOOGLE_PROJECT_ID"), dialect='standard')


Downloading: 100%|██████████|
data shape : (11441319, 11)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
event_id,object,0,0.000000,11441319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000533-3f1c-4b3b-81f1-0c8f35754b4e,1
event_datetime,datetime64[us],0,0.000000,1756107,2023-07-29 01:58:17.236437,NaN,2023-07-18 00:00:00,2023-07-22 21:31:17,2023-07-28 17:08:58,2023-08-04 17:59:10,2023-08-10 23:59:59,2023-08-06 20:22:22,172
event_key,object,0,0.000000,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,view_lab_tap,1266665
session_id,object,0,0.000000,253616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,8157
id,object,0,0.000000,11441319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000533-3f1c-4b3b-81f1-0c8f35754b4e,1
item_name,object,0,0.000000,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,11428280
page_name,object,0,0.000000,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,10652540
friend_count,float64,752556,6.577528,370,54.343567,33.507977,0.0,32.0,49.0,71.0,1365.0,0.0,260578
votes_count,float64,754554,6.594991,1499,257.274234,218.068191,0.0,97.0,210.0,362.0,3017.0,0.0,326024
heart_balance,float64,728643,6.368523,14503,16269.292655,3317340.091112,0.0,434.0,1249.0,3188.0,884999804.0,300.0,149106


Table: hackle_properties
Downloading: 100%|██████████|
data shape : (525350, 8)


,dtypes,missing,missing_rate (%),nunique,mean,std,min,25%,50%,75%,max,mode,mode_counts
id,Int64,0,0.0,525350,262675.5,151655.626297,1.0,131338.25,262675.5,394012.75,525350.0,1,1
session_id,object,0,0.0,253616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,040914e1-61ac-40ef-b76a-718066d880dc,9
user_id,object,0,0.0,327381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,82255
language,object,0,0.0,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ko-KR,340900
osname,object,0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iOS,359479
osversion,object,0,0.0,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.5.1,218699
versionname,object,0,0.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0.5,309644
device_id,object,0,0.0,251720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,040914e1-61ac-40ef-b76a-718066d880dc,9


# 의존성 체크

MYSQL Workbench를 통해 데이터의 관계도식을 살펴본 결과 다음과 같이 출력되었다.

<img src="./img/erd_screenshow.png">

- `hackle` 데이터베이스 구조는 관계구조가 존재하지 않음.
    - 데이터를 살펴보고 관계를 이어줄 수 있으면 이어줘야 한다고 생각.
    - 하지만 데이터를 살펴보면 어정쩡하게 관계가 나타나있어서 제대로 이어질 수 있을지는 미지수.
- `votes` 데이터베이스는 `accounts_userwithdraw` 테이블이 어떠한 votes 내 테이블과 관계도식이 그려져 있지 않음
    - 이는 다른 테이블을 통해 “어떠한 관계” 을 포함하는 내용을 포함하고 분석할 수 없다는 결론이 지어짐.
    - `accounts_userwithdraw`는 개별 테이블로 지정하고 그 테이블 내에서 요인분석을 통해 진행되어야 한다고 생각.

**질문 사항**

- `votes` 데이터베이스의 `accounts_user` 테이블의 `id`는 `hackle` 데이터베이스의 `hackle_properties` 테이블의 `user_id`와 의존 관계를 가지고 있는가?

In [ ]:
# votes.accounts_user 테이블과 hackle.hackleproperties 테이블의 의존성 체크
# 50000만개의 데이터를 통해서 부분체크
accounts_user_df = load_data_from_bigquery("SELECT * FROM `votes.accounts_user` LIMIT 50000;")
hackle_properties_df = load_data_from_bigquery("SELECT * FROM `hackle.hackle_properties` LIMIT 50000;")

# accounts_user_df와 hackle_properties_df의 의존성 체크
def check_dependency(accounts_user_df, hackle_properties_df):
    # accounts_user_df의 user_id와 hackle_properties_df의 user_id를 비교
    accounts_user_ids = set(accounts_user_df['id'].dropna().unique())
    hackle_properties_ids = set(hackle_properties_df['user_id'].dropna().unique())
    
    # 공통된 user_id 찾기
    common_ids = accounts_user_ids.intersection(hackle_properties_ids)
    
    print(f"Total unique user_ids in accounts_user: {len(accounts_user_ids)}")
    print(f"Total unique user_ids in hackle_properties: {len(hackle_properties_ids)}")
    print(f"Common user_ids: {len(common_ids)}")
    
    return common_ids

# 의존성 체크 실행
common_user_ids = check_dependency(accounts_user_df, hackle_properties_df)

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Total unique user_ids in accounts_user: 50000
Total unique user_ids in hackle_properties: 38573
Common user_ids: 0


두 데이터베이스의 의존성은 전혀 확인이 되지 않았음을 볼 수 있음.

즉, **독립된 데이터베이스** 인 것으로 확인 가능!

# 각 테이블 별 EDA

사용을 완료한 데이터는 `del`을 통하여 메모리 할당 제거

In [23]:
# 테이블 하나씩 불러오기 위한 함수 작성
def load_tables(table:str, dataset):
    if table in dataset['table_name'].values:
        seleted_table = table
        return seleted_table
    
    else:
        print(f"Table {table} not found in dataset.")
        return None

## votes 데이터베이스